In [4]:
# パッケージのimport
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Function
import torch.utils.data as data
import torch
import cv2
import numpy as np
import os.path as osp
import glob, pprint
from itertools import product as product
from math import sqrt as sqrt

# XMLをファイルやテキストから読み込んだり、加工したり、保存したりするためのライブラリ
import xml.etree.ElementTree as ET

# フォルダ「utils」のdata_augumentation.pyからimport。入力画像の前処理をするクラス
from utils.data_augumentation import Compose, ConvertFromInts, ToAbsoluteCoords, PhotometricDistort, Expand, RandomSampleCrop, RandomMirror, ToPercentCoords, Resize, SubtractMeans

# フォルダ「utils」にある関数matchを記述したmatch.pyからimport
from utils.match import match

In [2]:
root_path = "./data/IBEMdataset/Tr00/"

jpeg_path_template = osp.join(root_path + "*.jpg")
text_path_template = osp.join(root_path + "*.txt")

img_list = glob.glob(jpeg_path_template)
anno_list = glob.glob(text_path_template)

print(len(img_list))
print(len(anno_list))

4082
4082


In [19]:
# 「XML形式のアノテーション」を、リスト形式に変換するクラス

ind = 1

img = cv2.imread(img_list[ind])  # [高さ][幅][色BGR]
height, width, channels = img.shape

txt_path = anno_list[ind]

# 画像内の全ての物体のアノテーションをこのリストに格納します
ret = []

# txtファイルを読み込む
with open(txt_path, mode='rt') as f:
    anno_data = f.readlines()

for i in range(4, len(anno_data)):
    bndbox = anno_data[i].replace(' ', '').replace('\n', '').split("\t")
    for j in range(len(bndbox)):
        bndbox[j] = float(bndbox[j])

    # アノテーションの xmin, ymin, xmax, ymaxを取得し、0～1に規格化
    bbox_xywh = bndbox[:4]
    xmin = bbox_xywh[0]
    ymin = bbox_xywh[1]
    xmax = xmin + bbox_xywh[2]
    ymax = ymin + bbox_xywh[3]

    # アノテーションのクラスを追加(Inline:0, Display:1)
    bndbox = [xmin/100, ymin/100, xmax/100, ymax/100, int(bndbox[4])]

    ret += [bndbox]

pprint.pprint(ret)


[[0.2218, 0.5376, 0.3676, 0.5698, 1],
 [0.2218, 0.6221, 0.6972999999999999, 0.6607000000000001, 1],
 [0.22940000000000002, 0.1577, 0.2695, 0.1723, 0],
 [0.48450000000000004, 0.1582, 0.49690000000000006, 0.17190000000000003, 0],
 [0.6918000000000001, 0.5815, 0.7782000000000001, 0.5966, 0],
 [0.2979, 0.5986, 0.3387, 0.6137, 0],
 [0.6372, 0.6724, 0.6711, 0.6869999999999998, 0],
 [0.2274, 0.6729, 0.2502, 0.6866000000000001, 0],
 [0.5798, 0.6768000000000001, 0.5909, 0.6836000000000001, 0],
 [0.44780000000000003, 0.6904, 0.4644, 0.7007000000000001, 0],
 [0.557, 0.7406999999999999, 0.6897, 0.7558, 0],
 [0.7554000000000001, 0.7417, 0.7692, 0.7524, 0],
 [0.6068, 0.7583, 0.6406999999999999, 0.7728999999999999, 0]]


"\n        # 画像内にある物体（object）の数だけループする\n        for obj in xml.iter('object'):\n\n            # アノテーションで検知がdifficultに設定されているものは除外\n            difficult = int(obj.find('difficult').text)\n            if difficult == 1:\n                continue\n\n            # 1つの物体に対するアノテーションを格納するリスト\n            bndbox = []\n\n            name = obj.find('name').text.lower().strip()  # 物体名\n            bbox = obj.find('bndbox')  # バウンディングボックスの情報\n\n            \n            pts = ['xmin', 'ymin', 'xmax', 'ymax']\n\n            for pt in (pts):\n                # VOCは原点が(1,1)なので1を引き算して（0, 0）に\n                cur_pixel = int(bbox.find(pt).text) - 1\n\n                # 幅、高さで規格化\n                if pt == 'xmin' or pt == 'xmax':  # x方向のときは幅で割算\n                    cur_pixel /= width\n                else:  # y方向のときは高さで割算\n                    cur_pixel /= height\n\n                bndbox.append(cur_pixel)\n\n            \n            label_idx = self.classes.index(name)\n            bndbox.append(label_

In [1]:
a = [[] for i in range(5)]
print(a)

[[], [], [], [], []]
